# Segunda Avaliação Parcial

Neste módulo trabalhamos a visualização de dados, assim como as técnicas para acessar estes dados da internet. As técnicas de web scraping que estudamos, usando __Requests__, __Beautiful Soup__ e __Selenium__, permitem implementar aplicações que processam informação diretamente da rede.  

Desde a Instrução prática 7, por exemplo, estamos trabalhando com dados que baixamos do [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos).  

Aprendemos também como implementar uma aplicação com GUI utilizando o __TK Inter__. 

Neste contexto vamos implementar uma aplicação que, a traves de uma GUI, permita gerar gráficos de precipitações e temperaturas medias mensais de um determinado ano, para uma estação meteorológica. A atividade deve ser organizada da seguinte forma.

## Exercício 1

__Primeiro protótipo__: Desenvolva uma interface gráfica que pegue, em tempo real, as informações sobre os dados de quais anos estão disponíveis no site do [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos). Os anos disponíveis devem ser apresentados utilizando um widget apropriado que permita selecionar um, e apenas um ano. Se não tiver conexão a Internet ou se o site não estiver disponível o aplicativo deve mostrar uma mensagem de erro na interface.


A classe BuscaDados vai fazer a requisição para coletar a lista de anos.
O método obter_anos_disponivei é responsável por fazer uma solicitação HTTP ao URL e extrair informações sobre os anos disponíveis. Ele retorna uma tupla contendo a lista de anos e possíveis erros encontrados durante o processo.
Ele tenta fazer uma solicitação HTTP para o URL fornecido usando requests.get(). Se ocorrer algum erro HTTP, ele é capturado e armazenado em self.__erros. Se ocorrer qualquer outro tipo de exceção, também é capturado e armazenado em self.__erros.
Se nenhuma exceção for lançada durante a solicitação, o conteúdo da resposta é analisado usando BeautifulSoup para extrair os anos disponíveis e seus respectivos URLs. Essas informações são armazenadas na lista self.__anos.



A classe interface é responsável por criar uma interface gráfica usando a biblioteca Tkinter em Python para interagir com dados meteorológicos.
No método interface é criada uma janela de aplicativo Tkinter. Ele chama o método obter_anos_disponiveis da instância de BuscaDados para recuperar os anos disponíveis e possíveis erros. Com base nos resultados obtidos, a interface é construída de acordo com os anos disponíveis ou exibe uma mensagem de erro se houver problemas ao obter os dados.
Se os anos estiverem disponíveis, é criado um menu (OptionMenu) com os anos disponíveis.
e não for possível obter os anos, é exibida uma mensagem de erro na interface.
Finalmente, a interface entra em loop (mainloop) para que o aplicativo permaneça aberto e responsivo até que o usuário decida fechá-lo.

In [ ]:

class BuscaDados:
    
    def __init__(self, anos, erros):
        self.__anos = anos
        self.__erros = erros
    
    def obter_anos_disponiveis(self):
        
        self.__anos =  []
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        url = "https://portal.inmet.gov.br/dadoshistoricos"
        
        resposta = requests.get(url, headers=headers)  
        
        self.__erros= None
        try:
            resposta = requests.get(url, headers=headers)
            resposta.raise_for_status()
        except requests.HTTPError as http_err:
            self.__erros = http_err
        except Exception as err:
            self.__erros = err
        else:
            texto = BeautifulSoup(resposta.text, 'html.parser')
            articles = texto.find_all('article')
            for i in articles:
                ano_string = i.find('a').string
                ano_formatado = ''.join(filter(str.isdigit, ano_string))
                self.__anos.append((ano_formatado, i.find('a')['href']))
        
        return self.__anos, self.__erros


class Interface:
    
    def __init__(self):
        self.ano = BuscaDados([], erros = None)
        self.dados = ObterDados()
    
    def interface(self):
        app = tk.Tk()
        app.title("Dados Meteorológicos")
        app.geometry('500x300')
        app.configure(background='black')
        
        anos, erros = self.ano.obter_anos_disponiveis()  #atribuir valores retornados pela função
        self.anos = anos  # Atribuir anos à instância da classe

        if self.anos:
            opcao = tk.StringVar(app)
            opcao.set(self.anos[0][0])
            texto = tk.OptionMenu(app, opcao, *[ano[0] for ano in self.anos])
            texto.place(x=10, y=10, width=100, height=50)
        else:
            mensagemErro = tk.Label(app, text=f"Erro ao obter dados: {erros}", background='#c38680', wraplength=450)
            mensagemErro.place(x=10, y=70, width=450, height=30)

        app.mainloop()


In [ ]:
%run -m estacoes_V1

## Exercício 2

__Segundo protótipo__: Com base no ano escolhido sua aplicação deve baixar o arquivo compactado, com os dados de todas as estações, descompactar ele numa pasta temporária e mostrar em um outro widget a lista de estações disponíveis para selecionar uma e apenas uma delas. O widget do protótipo anterior deve ficar disponível para que o usuário possa trocar o ano se desejar. Nesse caso um novo arquivo será baixado e descompactado e as estações disponíveis exibidas para seleção.

Para a segunda aplicacao, foi add a classe ObterDados     
O método downloadDados procura o link correspondente ao ano selecionado na lista de anos fornecida. Se não encontrar o link, imprime uma mensagem de erro e retorna uma lista vazia.
Se o link for encontrado, o método cria um diretório para armazenar os arquivos extraídos, baixa o arquivo zip usando a biblioteca wget, extrai seu conteúdo usando zipfile.ZipFile e obtém os nomes das cidades dos arquivos extraídos.
Os nomes das cidades são extraídos dos títulos dos arquivos zip. Se ocorrer algum erro durante o processo de download ou extração, uma mensagem de erro é impressa e uma lista vazia é retornada.

In [ ]:
def escolhaCidades(self, anoSelecionado, app):     #mudança interface
        self.cidades = self.dados.downloadDados(anoSelecionado, self.anos)
        opcao2 = tk.StringVar(app)
        opcao2.set("CIDADE")
        texto2 = tk.OptionMenu(app, opcao2, *self.cidades)
        texto2.place(x=10, y=70, width=150, height=50)

class ObterDados:
    
     def downloadDados(self, anoSelecionado, anos):
        link = None
        for ano in anos:
            if ano[0] == anoSelecionado:
                link = ano[1]
                break  # Parar a busca se o ano for encontrado

        if link is None:
            print(f"Não foi possível encontrar o link para o ano {anoSelecionado}")
            return []  # Retorna uma lista vazia se o link não for encontrado

        caminho_zip = f'csv/{anoSelecionado}.zip'
        caminho_extracao = 'csv/'
        
        
        if not os.path.exists(caminho_extracao):
            os.makedirs(caminho_extracao)

        try:
            # Baixar o arquivo zip
            wget.download(link, caminho_zip)
            
            # Extrair o conteúdo do arquivo zip
            with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
                zip_ref.extractall(caminho_extracao)
                titulos_arquivos = zip_ref.namelist()
            
            # Extrair nomes das cidades dos títulos dos arquivos
            nomes_cidades = [titulo.split('_')[4] for titulo in titulos_arquivos if len(titulo.split('_')) >= 5]

            return nomes_cidades
        except Exception as e:
            print(f"Erro ao baixar/descompactar arquivos: {e}")
            return []  # Retorna uma lista vazia em caso de erro

In [ ]:
%run -m estacoes_V2

# Exercício 3

__Terceiro protótipo__: Com base na estação selecionada gere e exiba dentro da aplicação os gráficos de precipitações e temperaturas medias mensais para a estação selecionada. A interface deve permitir que o usuário troque a estação selecionada a qualquer momento, assim como foi feito co o ano, gerando novos gráficos.   

È feita uma nova classe para criar os dataFrame. 

Método dataFrame: Este método recebe uma cidade, um ano selecionado e uma lista de anos. Ele chama o método downloadDados da instância de ObterDados para obter o caminho do arquivo CSV correspondente à cidade e ao ano selecionados. Em seguida, lê o arquivo CSV usando pd.read_csv, define o índice como uma combinação da data e hora e trata dados ausentes.

Método tratamentoDados: Este método realiza tratamentos nos dados do DataFrame, substituindo valores de precipitação e temperatura que ultrapassam determinado limite por None.

Método plotGraficos: Este método cria dois subplots, um para a temperatura média mensal e outro para a precipitação média mensal. Ele calcula as médias mensais para temperatura e soma das precipitações, plota os dados usando plt.plot e ajusta as configurações dos gráficos.

Função plotGrafico: Esta função está fora da classe e é usada para plotar os gráficos. Ela cria uma instância de Graficos, chama os métodos dataFrame, tratamentoDados e plotGraficos para gerar os gráficos com base nos parâmetros cidade, ano selecionado e lista de anos.

In [ ]:
def grafico(self,cidade, anoSelecionado, anos,win):    #mudança em interface
        Graficos.plotGrafico(cidade, anoSelecionado, anos)
        canvas = FigureCanvasTkAgg(plt.gcf(), master=win)
        canvasWidget = canvas.get_tk_widget()
        canvasWidget.place(x=180, y=10, width=500, height=400)

class Graficos:
    def __init__(self):
        self.dow = obtencaoDados.ObterDados()
        self.cidade = None
        self.anoSelecionado = None
        self.__df = None
        self.caminhoCSV = None
    
    def dataFrame(self, cidade, anoSelecionado,anos):
        self.dow = obtencaoDados.ObterDados()
        titulos_arquivos = self.dow.downloadDados(anoSelecionado, anos)[1]
        for titulos in titulos_arquivos:
            if cidade in titulos:

                self.caminhoCSV = f'csv/{titulos}' # Atribuir valor a caminhoCSV
          
            

        self.__df = pd.read_csv(self.caminhoCSV, encoding="iso-8859-1", decimal=',', sep=';', skiprows=8)
        self.__df.index = pd.DatetimeIndex(self.__df["DATA (YYYY-MM-DD)"]+" "+self.__df["HORA (UTC)"], name="DATA") 
    
        if self.caminhoCSV is None:
            print(f"Arquivo CSV não encontrado para a cidade {cidade} no ano {anoSelecionado}")
            return  # Sair da função se o caminhoCSV não foi definido

    def tratamentoDados(self):
        self.__df.loc[self.__df["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].abs() > 1000, "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"] = None
        self.__df.loc[self.__df["TEMPERATURA DO PONTO DE ORVALHO (°C)"].abs() > 1000, "TEMPERATURA DO PONTO DE ORVALHO (°C)"] = None
    
    
    def plotGraficos(self):
        df = self.__df.groupby(self.__df.index.month)["TEMPERATURA DO PONTO DE ORVALHO (°C)"].mean()
        plt.clf()
        plt.subplot(2,1,1)
        plt.plot(df.index, df, marker='s')
        plt.title('Temperatura Média Mensal')
        plt.xlabel('Mês')
        plt.ylabel('Temperatura (°C)')
        plt.grid(True)

        dfT = self.__df.groupby(self.__df.index.month)["PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"].sum()
        plt.subplot(2,1,2)
        plt.plot(dfT.index, dfT, marker='s')
        plt.title('Precipitação Média Mensal')
        plt.xlabel('Mês')
        plt.ylabel('Precipitação')
        plt.grid(True)
        plt.subplots_adjust(wspace=0,hspace=1)
        
    def plotGrafico(cidade,anoSelecionado,anos):
        df = Graficos()
        df.dataFrame(cidade,anoSelecionado,anos)
        df.tratamentoDados()
        df.plotGraficos()
    

In [ ]:
%run -m estacoes_V3

## Respostas

Implemente sua aplicação como um pacote __Python__ e disponibilize neste __Notebook__ um explicação dos principais pontos da implementação. Utilize o comando mágico ``%run`` para executar o aplicativo desenvolvido.  Envie o __Notebook__ via __Moodle__ assim como o repositório com a implementação do aplicativo até o final do prazo. 